# Notebook facultatif : préparation du jeu de données à partir des données brutes

Quand on se lance avec enthousiasme dans un nouveau projet de machine-learning, on pense avant tout au choix du modèle que l'on va utiliser, ce qui constitue la partie "noble" de l'étude. 

Cependant, et même s'il ne s'agit pas de l'étape la plus agréable du travail, un préalable indispensable est de réunir les données brutes puis de les mettre en forme pour qu'elles puissent être ingérées par le modèle.

L'objectif de ce TP est d'effectuer ce travail de construction du jeux de données "propres" à partir de données brutes issues de sources diverses.

Nos fichiers d'entrée bruts sont les suivants :
> * historique de consommation nationale (source : Eco2mix) : **conso_2014_2018.csv**
> * calendrier des jours fériés : **jours_feries.csv**
> * coordonnées géographiques et les poids associés aux stations météo France : **stations_meteo_RTE.csv**
> * historique d'observations et de prévisions à 24h de température : **temperatures.csv**

Et les fichiers que l'on va créer sont :
> * **x_input_full.csv**  # Les entrées pour le modèle d'apprentissage
> * **x_input_light.csv**  # Une version allégée de *x_input_full*
> * **y_conso.csv**  # les sorties pour le modèle d'apprentissage

## Mise en place de l'environnement

Exécutez la cellule ci-dessous, par exemple avec shift-entrée.

In [ ]:
# Chargement des bibliothèques python, et quelques éléments de configuration. 
# Si vous exécuter ce notebook depuis votre PC, il faudra peut-etre installer certaines librairies avec   
# "pip3 install ma_bibliotheque"
import os  # accès aux commandes système
import datetime  # structure de données pour gérer des objets calendaires
import pandas as pd  # gérer des tables de données en python
import numpy as np  # librairie d'opérations mathématiques
import zipfile  # manipulation de fichiers zip
from sklearn.preprocessing import StandardScaler  # normalisation des données
import joblib  # sauvegarde du scaler

In [ ]:
data_folder = os.path.join(os.getcwd(), "data")
raw_data_folder = os.path.join(data_folder, "raw_data")
transformed_data_folder = os.path.join(data_folder, "transformed_data")

print("")
print("Mon repertoire de raw_data_folder est : {}".format(raw_data_folder))
print("")
print("Fichiers/dossiers contenus dans ce repertoire :")
for file in os.listdir(raw_data_folder):
    print(" - " + file)    

## import en mémoire des données

### Données de consommation

Dans un premier temps on importe les données de consommation réalisée à partir du fichier **conso_2014_2018.csv**.   
La date et l'heure sont données dans la première colonne, la consommation nationale dans la suivante.

In [ ]:
# Les données du csv sont importées dans un objet de type DataFrame
conso_csv = os.path.join(raw_data_folder, "conso_2014_2018.csv")
conso_df = pd.read_csv(conso_csv, sep=",")

Vérifions que les données ont été importées correctement :

In [ ]:
# Afficher les dimensions et le noms des colonnes de la data frame
print(conso_df.shape)  # Nombre de lignes, nombre de colonnes
display(conso_df.head(2))
display(conso_df.tail(2))

#### Petit détour pour gérer les dates

Le fichier **conso_2014_2018.csv** contient en particulier la colonnes **ds**. Celle-ci contient une variable de type *string* correspondant à la date et à l'heure. 

<img src="pictures/clock.png" width=60 height=60>

Pour manipuler facilement ces variables, il est recommandé de les convertir en objet de type **datetime**.

In [ ]:
conso_df['ds'] = pd.to_datetime(conso_df['ds'], utc=True)

In [ ]:
display(conso_df.head(2))

La cellule ci-dessous a pour but d'illustrer comment utiliser ces objets **datetime**.  
Ces quelques lignes de code pourraient vous reservir ultérieurement.

In [ ]:
# datetime vers string
noel_2017_date = datetime.date(2017, 12, 25)
noel_2017_str = datetime.datetime.strftime(noel_2017_date, format="%Y-%m-%d")
print("noel_2017_date vaut : {}, et est de type {}".format(noel_2017_date, str(type(noel_2017_date))))
print("noel_2017_str vaut : {}, et est de type {}".format(noel_2017_str, str(type(noel_2017_str))))
print("---")

# string vers datetime
starwars_day_2017_str = "2017-05-04"
starwars_day_2017_date = datetime.datetime.strptime(starwars_day_2017_str, "%Y-%m-%d")
print("starwars_day_2017_date vaut : {}, et est de type {}".format(starwars_day_2017_date, str(type(starwars_day_2017_date))))
print("D'ailleurs, c'était le " + str(starwars_day_2017_date.weekday() + 1) + " ème jour de la semaine, où 0 correspond à lundi et 6 correspond à dimanche")
print("starwars_day_2017_str vaut : {}, et est de type {}".format(starwars_day_2017_str, str(type(starwars_day_2017_str))))
print("---")

# Voyager dans le temps
saint_sylvestre_2017_date = datetime.date(2017, 12, 31)
bienvenu_en_2018_date = saint_sylvestre_2017_date + datetime.timedelta(days=1)
print("Le 31 décembre 2017 plus un jour ça donne le {}".format(bienvenu_en_2018_date))

### Récuperation des jours fériés

Le fichier **jours_feries.csv** contient la liste des jours fériés. 

In [ ]:
jours_feries_csv = os.path.join(raw_data_folder,"jours_feries.csv")
jours_feries_df = pd.read_csv(jours_feries_csv, sep=";")

jours_feries_df.ds = pd.to_datetime(jours_feries_df.ds, utc=True)  # comme ci-dessus

jours_feries_df.head(3)

### Récupération des coordonnées géographiques des stations météo

Dans le but d'obtenir un modèle de machine-learning performant, nous allons créer une variable artificielle qui sera représentative de la température nationale France. Cette variable n'aura été relevée par aucun thermomètre, mais sera un barycentre des températures mesurées aux différentes stations météo. Chaque station météo s'est vue attribuer un poids permettant de sentir sa contribution. Pour en savoir plus sur les poids :  
https://clients.rte-france.com/lang/fr/visiteurs/services/actualites.jsp?id=9482&mode=detail

Commençons par charger le csv qui à chaque station météo attribue sa longitude/latitude/poids.

In [ ]:
stations_meteo_csv = os.path.join(raw_data_folder, "stations_meteo_RTE.csv")
stations_meteo_df = pd.read_csv(stations_meteo_csv, sep=";")

In [ ]:
stations_meteo_df.head(3)

In [ ]:
# Pour compter le nombre de stations il suffit de compter le nombre de lignes dans le data-frame
nb_stations = stations_meteo_df.shape[0]
print(nb_stations)

In [ ]:
# Verifions que la somme des poids fait 1
np.sum(stations_meteo_df["Poids"])

### Récupération des températures

<img src="pictures/weather.png" width=60 height=60>

On va utiliser les mêmes fonctions que précédemment pour lire le fichier **temperatures.csv**, qui contient les historiques des température réalisées et prévues pour différentes stations météo France.

**Attention : Les données météo sont encryptées dans un fichier zip.**  
Pour les lire vous avez besoin d'un mot de passe qui ne peut vous être donné que dans le cadre d'un travail au sein de RTE.

In [ ]:
meteo_zip = os.path.join(raw_data_folder, "temperatures.zip")

In [ ]:
password = ""

In [ ]:
# Cette étape peut être un peu longue car le fichier est volumineux
zip_file_meteo = zipfile.ZipFile(meteo_zip)
zip_file_meteo.setpassword(bytes(password, 'utf-8'))
meteo_df = pd.read_csv(zip_file_meteo.open('temperatures.csv'), sep=",", engine='c', header=0)
meteo_df['ds'] = pd.to_datetime(meteo_df.date_cible, utc=True)
meteo_df = meteo_df.drop(columns=['date_cible'])

In [ ]:
print(meteo_df.shape)  # (nb lignes , nb_colonnes)
print(meteo_df.columns)

In [ ]:
display(meteo_df.head(3))
display(meteo_df.tail(3))

On remarque que les données s'arrête au 21/12/2018 et que pour cette date, on ne dispose que des prévisions à 24h, pas des réalisations.

### Une variable complémentaire à considérer : les données TEMPO

Pour l'exemple de la prévision de consommation, il serait pertinent de fournir en entrée du modèle l'information sur le type de jour Tempo. Les clients ayant souscrit à ce type de contrat sont incités à réduire leur consommations les jours BLANC et ROUGE, aussi on peut penser que cette information permettra d'améliorer la qualité des prédictions.

Elles ne sont pas utilisées dans le cadre de ce TP, mais le stagiaire motivé pourra trouver des informations complémentaires sur le web :  
http://www.rte-france.com/fr/eco2mix/eco2mix-telechargement

##  Fusion des données

<img src="pictures/fusion.png" width=600 height=200>

On va maintenant construire un dataframe unique qui regroupe toutes les données nécessaire à notre modèle de prévision. On aura ici une ligne pour chaque timestamp, et dans cette ligne à la fois notre X et notre Y pour le futur modèle de machine-learning.

### Fusion des données de consommation et de jours fériés

**Attention**  
L'heure n'apparaît pas dans *jours_feries_df*, mais implicitement, elle vaut 00:00 pour toutes les observations.  
Il faut y prendre garde lors de la la fusion avec les données de consommation, afin que le point horaire de 15h35 d'un jour férié soit bien reconnu comme appartenant à un jour férié. Pour cela, nous allons créer dans les deux jeux de données une colonne **day**, qui ne conserve que la date. Nous utiliserons ensuite cette colonne pour fusionner les deux jeux de données.

In [ ]:
# On rajoute la colonne *day* aux 2 dataframes
conso_df['day'] = conso_df['ds'].apply(lambda x: datetime.datetime.strftime(x, format="%Y-%m-%d"))
jours_feries_df['day'] = jours_feries_df['ds'].apply(lambda x: datetime.datetime.strftime(x, format="%Y-%m-%d"))

# On merge sur cette colonne
merged_df = pd.merge(conso_df, jours_feries_df, on='day', how="left", suffixes=("", "_tmp"))

# On peut maintenant supprimer ces colonnes temporaires
merged_df = merged_df[["ds", "conso_real", "holiday"]]
display(merged_df.loc[0:24])

In [ ]:
merged_df["is_bank_holiday"] = merged_df["holiday"].astype("str").apply(lambda x: 0 if x == "nan" else 1)
merged_df.drop(['holiday'], axis=1, inplace=True)

# Regardons
display(merged_df.loc[0:24])

### Ajout des informations de température.

In [ ]:
merged_df = pd.merge(merged_df, meteo_df, on = 'ds')

In [ ]:
print(merged_df.shape)
print(merged_df.columns)

### Calcul de la variable artificielle "température France"

On va ajouter une colonne à notre dataframe, colonne que - par expérience/expertise - on sait pouvoir être utile pour prévoir la consommation.

La température France est une moyenne pondérée de la température de 32 stations. C'est ici que nous avons besoin des poids de *stations_meteo_df*.

In [ ]:
merged_df['temperature_real'] = np.dot(merged_df[list(merged_df.columns[merged_df.columns.str.endswith("_0")])], stations_meteo_df['Poids'])
merged_df['temperature_prevue'] = np.dot(merged_df[list(merged_df.columns[merged_df.columns.str.endswith("_24")])], stations_meteo_df['Poids'])

In [ ]:
print(merged_df.shape)
print(merged_df.columns)
display(merged_df.head(2))

### Ajout des variables retardées

Le modèle prédictif n'aura pas le droit d'avoir accès à la température réalisée du futur (dommage, ça rendrait notre métier plus facile :-)), mais en revanche nous avons accès à la connaissance de la température réalisée 24 heures avant.

De même pour la consommation.

Ainsi, nous avons accès au réalisé (température, conso nationale) du J-1, et nous avons une température prévue pour le point horaire cible. Charge à notre modéle de prévision de nous proposer une valeur d ela consommation nationale pour le point horaire cible.

In [ ]:
merged_df["temperature_real_24h_avant"] = merged_df["temperature_real"].shift(24)

In [ ]:
merged_df["conso_real_24h_avant"] = merged_df["conso_real"].shift(24)

<font color='blue'>

**Prenons quelques instants pour comprendre ces histoires de variables retardées.**

Imaginons que l'on veuille prédire la consommation nationale pour le point horaire cible du 09/01/2014 à 00h00 (ligne 24 de working_df)

Utiliser dans un modèle de prédiction :
* la consommation réalisée de la veille (08/01/2014 à 00h00)
* les infos de météo réalisée de la veille
* les **prévisions** météo du point horaire cible (09/01/2014 à 00h00)

semble être pertinent.

</font>
<br/>
  
<font color='green'>

* Est-ce que le shift a été fait dans le bon sens ?

</font>

## Supression des variables inutiles

In [ ]:
merged_df = merged_df.drop("temperature_real", axis=1)
merged_df = merged_df.drop([x for x in merged_df.columns if (x.endswith("_0") or x.endswith("_24"))], axis=1)

print(merged_df.shape)
print(merged_df.columns)
display(merged_df.head(2))

## Suppression des NaN

On souhaite avoir un jeu de données complet : pas de trous dans les dates et pour chaque point horaire, toutes les informations disponibles (consommation, température réalisée et prévue).

On affiche donc les dates pour lesquelles il manque des informations et on conservera un intervalle ne contenant pas ces dates.

In [ ]:
mask = merged_df.isnull().any(axis=1)

print(np.sum(mask))
print(merged_df.index[mask].tolist())

En début de dataset, avec nos manipulation pour obtenir les valeurs de la veille, il est normal que l'on ait quelques *NaN* pour les colonnes 24h_avant.

Quant à la ligne d'index 165, bah... C'est la vie.

maintenant nous n'allons garder que les lignes sans *NaN*.

In [ ]:
merged_df = merged_df[~mask]
merged_df.reset_index(drop=True, inplace=True)  # on demande de remettre à jour les numéros d'index de la DF pour commencer à 0

print(merged_df.shape)
print(merged_df.columns)
display(merged_df.head(2))

### Normalisation ("scaling")

In [ ]:
columns_to_scale = ["conso_real", "temperature_prevue", "temperature_real_24h_avant", "conso_real_24h_avant"]
new_column_names = [x + "_scaled" for x in columns_to_scale]

scaler = StandardScaler()
scaler.fit(merged_df[columns_to_scale])
scaled_values = scaler.transform(merged_df[columns_to_scale])
scaled_values_df = pd.DataFrame(scaled_values, columns=new_column_names)

In [ ]:
# Remarque en passant:
# Ne pas relancer cette cellule, sinon
# working_df va beaucoup grossir...
merged_df = pd.concat([merged_df, scaled_values_df], axis=1)

In [ ]:
display(merged_df.head(2))
print(merged_df.shape)

# Petite verif manuelle
mean = np.mean(merged_df["conso_real"])
mean_scaled = np.mean(merged_df["conso_real_scaled"])
print()
print("--- Petite verif ---")
print("Moyenne de conso_real: " + str(mean))
print("Moyenne de conso_real_scaled: " + str(mean_scaled))

In [ ]:
# Pour plus tard, on se garde de côté un scaler uniquement dénormaliser la prédiction de conso
scaler_conso_nat = StandardScaler()
scaler_conso_nat.fit(merged_df[["conso_real"]])

# Verif
scaler_conso_nat.inverse_transform(merged_df["conso_real_scaled"])

### Encoding de la date

In [ ]:
merged_df['month'] = merged_df['ds'].dt.month
merged_df['hour'] = merged_df['ds'].dt.hour

# On sépare les jours de la semaine en week-end / pas week-end
# De base, la fonction datetime.weekday() renvoie 0 => Lundi, 2 => Mardi, ..., 5 => Samedi, 6 => Dimanche
# Ci-dessous, si on a un jour de la semaine alors dans la colonne weekday on mettra 1, et 0 si c'est le week-end
merged_df['weekday'] = (merged_df['ds'].dt.weekday < 5).astype(int)  # conversion bool => int

# Regardons
merged_df.loc[[0, 1, 23, 24, 24 *7 -1, 24 * 7, 1100], :]

Nous allons maintenant effectuer un **"one-hot encoding"** pour convertir ces informations sous un format utilisable par un réseau de neurones.

In [ ]:
encoded_month = pd.get_dummies(merged_df['month'], prefix="month")
encoded_hour = pd.get_dummies(merged_df['hour'], prefix="hour")

display(encoded_month.head(3))
display(encoded_hour.head(3))

In [ ]:
# Ne calculer cette cellule qu'une seule fois à cause du concat !
merged_df = pd.concat([merged_df, encoded_month, encoded_hour], axis=1)
merged_df = merged_df.drop(['month', 'hour'], axis=1)

In [ ]:
print(merged_df.shape)
print(merged_df.columns)
display(merged_df.head(2))

## Sauvegarde du fichier 

Tout d'abord on sépare les données en deux : 
- le vecteur de consommation à prévoir : y_conso
- La matrice des variables explicatives : x_input

Sachant que plus tard notre modèle aura pour mission d'établir une correspondance _f_ telle que l'on ait du mieux possible une relation *y = f(X)*.

In [ ]:
y_columns = ["ds", "conso_real", "conso_real_scaled"]

x_columns = ["ds", "is_bank_holiday"] \
+ ["temperature_real_24h_avant", 'temperature_real_24h_avant_scaled'] \
+ ["temperature_prevue", 'temperature_prevue_scaled'] \
+ ["conso_real_24h_avant", 'conso_real_24h_avant_scaled'] \
+ ["month_" + str(x) for x in range(1, 13)] \
+ ["hour_" + str(x) for x in range(0, 24)] \
+ ["weekday"]

In [ ]:
y = merged_df[y_columns]

In [ ]:
x = merged_df[x_columns]

In [ ]:
x.to_csv(os.path.join(transformed_data_folder,"x.csv"), index=False, sep=";")
y.to_csv(os.path.join(transformed_data_folder,"y.csv"), index=False, sep=";")

In [ ]:
joblib.dump(scaler_conso_nat, os.path.join(transformed_data_folder, "scaler_conso.save"))

Et enfin on zip **x.csv** avec un mot de passe car ce fichier contient les infos de température.

Depuis un terminal :
> zip -e x.zip x.csv  
> rm x.csv